In [1]:
import tensorflow as tf
print(tf.__version__)
import tensorflow_recommenders as tfrs
print(tfrs.__version__)

import math
@tf.function
def decode_fn(csv_line):
    defaults = [tf.constant(0, dtype=tf.int64),
           tf.constant(0, dtype=tf.int64),
           tf.constant(0, dtype=tf.float32),
           tf.constant(0, dtype=tf.int64)]
    csv_row = tf.io.decode_csv(csv_line, defaults)
    features = {}
    features["userId"] = csv_row[0]
    features["movieId"] = csv_row[1]
    labels = {
        "label": csv_row[2]
    }
    return (features, labels)
ctx = None # will not be none in distributed strategy, see later
def make_dataset_fn(path):
    dataset = tf.data.TextLineDataset([path], compression_type="GZIP")
    dataset = dataset\
        .batch(16, drop_remainder=True)\
    .map(decode_fn)
    return dataset
filenames = tf.data.Dataset.list_files("gs://mgaiduk-us-central1/ratings/csv_gzip/part*", shuffle=True, seed=42)
if ctx and ctx.num_input_pipelines > 1:
    filenames = filenames.shard(ctx.num_input_pipelines, ctx.input_pipeline_id)
dataset = filenames.interleave(make_dataset_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE, deterministic=False, cycle_length=8)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
for elem in dataset:
    break
elem

2023-03-07 13:46:14.943065: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-07 13:46:15.038856: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-07 13:46:15.042080: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-07 13:46:15.042115: I tensorflow/compiler/xla/stream_executor/cuda/cudar

2.11.0
v0.7.3


2023-03-07 13:46:16.136767: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-07 13:46:16.136799: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-07 13:46:16.136814: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mgaiduk-cpu-vm): /proc/driver/nvidia/version does not exist
2023-03-07 13:46:16.137064: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


({'userId': <tf.Tensor: shape=(16,), dtype=int64, numpy=
  array([   5,  154,  226,  600,  689,  717,  810,  909, 1208, 1409, 1847,
         2071, 2105, 2177, 2429, 2476])>,
  'movieId': <tf.Tensor: shape=(16,), dtype=int64, numpy=
  array([  191, 69526,   736,   435,  7759,  1240,  2087,  1527,   158,
          1722,  1037,   480,  1261,  6290,    39,  4037])>},
 {'label': <tf.Tensor: shape=(16,), dtype=float32, numpy=
  array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        dtype=float32)>})

In [3]:
vocab_size = 1000
hashing_layer = tf.keras.layers.experimental.preprocessing.Hashing(num_bins=vocab_size)
hashing_layer(elem[0]["userId"]) # it's good that we have an eager tensor on our hands, so we can test inputs-outputs to all parts of our model, heh?

<tf.Tensor: shape=(16,), dtype=int64, numpy=
array([971, 515, 509, 521, 490, 299, 925, 674, 347,  45, 252, 902, 563,
       215, 886, 238])>

In [4]:
embedding_dim = 8
lr = 0.01
initializer = tf.initializers.TruncatedNormal(
    mean=0.0, stddev=1 / math.sqrt(embedding_dim)
)
embedding_layer_feature_config = {
    "userId": tf.tpu.experimental.embedding.FeatureConfig(
        table=tf.tpu.experimental.embedding.TableConfig(
        vocabulary_size=vocab_size,
        initializer=initializer,
        dim=embedding_dim)),
    "movieId": tf.tpu.experimental.embedding.FeatureConfig(
        table=tf.tpu.experimental.embedding.TableConfig(
        vocabulary_size=vocab_size,
        initializer=initializer,
        dim=embedding_dim)),
}
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = lr)
embedding_layer = tfrs.layers.embedding.TPUEmbedding(
    feature_config=embedding_layer_feature_config,
    optimizer=optimizer)
hashed_tensor = {
    "userId": hashing_layer(elem[0]["userId"]),
    "movieId": hashing_layer(elem[0]["movieId"])
}
embeddings = embedding_layer(hashed_tensor)
embeddings

/home/maksimgaiduk/.local/lib/python3.9/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


{'userId': <tf.Tensor: shape=(16, 8), dtype=float32, numpy=
 array([[ 0.41343534, -0.31794858,  0.01400006, -0.1614508 , -0.3027821 ,
         -0.2894226 ,  0.24812186, -0.28222725],
        [ 0.08887184, -0.33360675,  0.45120153,  0.6819348 , -0.08404738,
         -0.14298265,  0.05770994, -0.5238398 ],
        [-0.5518302 , -0.41785422,  0.19948402,  0.18964908, -0.13487418,
         -0.2010811 ,  0.3907623 ,  0.13097371],
        [ 0.20046298,  0.20850155, -0.09723332, -0.15830232,  0.3276058 ,
          0.2102013 , -0.25354075,  0.07657696],
        [-0.12120442, -0.2367408 , -0.45796743, -0.29654923, -0.276082  ,
         -0.41840187,  0.2705518 , -0.19380979],
        [-0.2709396 ,  0.37509307, -0.20403184, -0.04137465,  0.25251728,
         -0.08419357,  0.2092655 , -0.19470377],
        [ 0.06902198,  0.08367513, -0.34424788,  0.28026772, -0.06520353,
          0.10085491, -0.39658692,  0.03365644],
        [ 0.20106868, -0.28069216, -0.07548938,  0.23359123, -0.00303984,
     

In [6]:
user_emb = embeddings["userId"]
movie_emb = embeddings["movieId"]
user_final_emb = tf.slice(user_emb, begin=[0, 0], size=[user_emb.shape[0],  user_emb.shape[1] - 1])
user_final_bias = tf.slice(user_emb, begin=[0, user_emb.shape[1] - 1], size=[user_emb.shape[0],  1])
movie_final_emb = tf.slice(movie_emb, begin=[0, 0], size=[movie_emb.shape[0],  movie_emb.shape[1] - 1])
movie_final_bias = tf.slice(movie_emb, begin=[0, movie_emb.shape[1] - 1], size=[movie_emb.shape[0],  1])
out = tf.keras.backend.batch_dot(user_final_emb, movie_final_emb) + user_final_bias + movie_final_bias
out

<tf.Tensor: shape=(16, 1), dtype=float32, numpy=
array([[-0.01751116],
       [ 0.12280549],
       [ 0.6105509 ],
       [ 0.23486945],
       [-1.1786304 ],
       [-0.04545186],
       [-0.23374282],
       [ 0.25991398],
       [ 0.14214638],
       [-0.02791688],
       [-0.05103706],
       [ 0.06510222],
       [ 0.05184928],
       [-0.6423423 ],
       [-0.40173113],
       [-0.3374161 ]], dtype=float32)>

In [7]:
class Model(tfrs.models.Model):
    def __init__(self):
        super().__init__()
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.BinaryCrossentropy(
                reduction=tf.keras.losses.Reduction.NONE
            ),
            metrics=[
                tf.keras.metrics.BinaryCrossentropy(name="label-crossentropy"),
                tf.keras.metrics.AUC(name="auc"),
                tf.keras.metrics.AUC(curve="PR", name="pr-auc"),
                tf.keras.metrics.BinaryAccuracy(name="accuracy"),
            ],
            prediction_metrics=[
                tf.keras.metrics.Mean("prediction_mean"),
            ],
            label_metrics=[
                tf.keras.metrics.Mean("label_mean")
            ]
        )
        self.optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = lr)
        self.hashing_layer = tf.keras.layers.experimental.preprocessing.Hashing(num_bins=vocab_size)
        embedding_layer_feature_config = {
            "userId": tf.tpu.experimental.embedding.FeatureConfig(
                table=tf.tpu.experimental.embedding.TableConfig(
                vocabulary_size=vocab_size,
                initializer=initializer,
                dim=embedding_dim)),
            "movieId": tf.tpu.experimental.embedding.FeatureConfig(
                table=tf.tpu.experimental.embedding.TableConfig(
                vocabulary_size=vocab_size,
                initializer=initializer,
                dim=embedding_dim)),
        }
        self.embedding_layer = tfrs.layers.embedding.TPUEmbedding(
            feature_config=embedding_layer_feature_config,
            optimizer=self.optimizer)
        self.final_activation = tf.keras.layers.Activation('sigmoid')
        

    def call(self, inputs):
        hashed_inputs = {}
        for field in ["userId", "movieId"]:
            hashed_inputs[field] = self.hashing_layer(inputs[field])
        print("Hashed inputs: ", hashed_inputs)
        embeddings = self.embedding_layer(hashed_inputs)
        user_emb = embeddings["userId"]
        movie_emb = embeddings["movieId"]
        # last unit of embedding is considered to be bias
        # out = tf.keras.backend.batch_dot(user_final[:, :-1], post_final[:, :-1]) + user_final[:, -1:] +  post_final[:, -1:]
        # This tf.slice code helps get read of "WARNING:tensorflow:AutoGraph could not transform ..." warnings produced by the above line
        # doesn't seem to improve speed though
        user_final_emb = tf.slice(user_emb, begin=[0, 0], size=[user_emb.shape[0],  user_emb.shape[1] - 1])
        user_final_bias = tf.slice(user_emb, begin=[0, user_emb.shape[1] - 1], size=[user_emb.shape[0],  1])
        movie_final_emb = tf.slice(movie_emb, begin=[0, 0], size=[movie_emb.shape[0],  movie_emb.shape[1] - 1])
        movie_final_bias = tf.slice(movie_emb, begin=[0, movie_emb.shape[1] - 1], size=[movie_emb.shape[0],  1])
        out = tf.keras.backend.batch_dot(user_final_emb, movie_final_emb) + user_final_bias + movie_final_bias
        prediction = self.final_activation(out) 
        return {
            "label": prediction
        }
    def compute_loss(self, inputs, training=False):
        features, labels = inputs
        outputs = self(features, training=training)
        # loss = tf.reduce_mean(label_loss)
        loss = self.task(labels=labels["label"], predictions=outputs["label"])
        print(loss)
        loss = tf.reduce_mean(loss)
        return loss

In [11]:
strategy = tf.distribute.get_strategy()
with strategy.scope():
    model = Model()
    model.compile(model.optimizer, steps_per_execution=10)

In [12]:
with strategy.scope():
    model.fit(dataset, epochs=1, steps_per_epoch=1000)

Hashed inputs:  {'userId': <tf.Tensor 'model/hashing_1/Identity:0' shape=(16,) dtype=int64>, 'movieId': <tf.Tensor 'model/hashing_1/Identity_1:0' shape=(16,) dtype=int64>}
Tensor("ranking/Identity:0", shape=(), dtype=float32)
Hashed inputs:  {'userId': <tf.Tensor 'while/model/hashing_1/Identity:0' shape=(16,) dtype=int64>, 'movieId': <tf.Tensor 'while/model/hashing_1/Identity_1:0' shape=(16,) dtype=int64>}
Tensor("while/ranking/Identity:0", shape=(), dtype=float32)
Hashed inputs:  {'userId': <tf.Tensor 'model/hashing_1/Identity:0' shape=(16,) dtype=int64>, 'movieId': <tf.Tensor 'model/hashing_1/Identity_1:0' shape=(16,) dtype=int64>}
Tensor("ranking/Identity:0", shape=(), dtype=float32)
Hashed inputs:  {'userId': <tf.Tensor 'while/model/hashing_1/Identity:0' shape=(16,) dtype=int64>, 'movieId': <tf.Tensor 'while/model/hashing_1/Identity_1:0' shape=(16,) dtype=int64>}
Tensor("while/ranking/Identity:0", shape=(), dtype=float32)
1000/1000 [==============================] - 2s 2ms/step - l